In [97]:
import sys
sys.path.append('../')
from pychemkin import ChemSolver, chemkin, InputParser, ReactionCoeffs,BackwardCoeffs
import numpy as np
import time

In [106]:
from scipy.integrate import solve_ivp, ode
x_init = np.ones(4)*1.e-10
T = 1000
chem = chemkin("../tests/test_xml/rxns_singlereversible.xml")
cs = ChemSolver(chem)
_,cs.kf,cs.kb = cs.chem._progress_rate_init(x_init,T)
cs.nu_diff_matrix = cs.chem.nu_prod-cs.chem.nu_react
print(solve_ivp(cs._ChemSolver__dy_dt, 
          t_span=(0, 6e-5), 
          y0=x_init, 
          t_eval=np.linspace(0,6e-5,20), method = 'LSODA'))

print("Using scipy.integrate.ode")

r = ode(cs._ChemSolver__dy_dt).set_integrator('lsoda')
r.set_initial_value(x_init,0)
t1 = 6.0e-5
dt = 3.15789474e-06
while r.successful() and r.t <= t1:
    print(r.t+dt, r.integrate(r.t+dt))

Finished reading xml input file
  message: 'The solver successfully reached the interval end.'
     nfev: 9435
     njev: 159
      nlu: 159
      sol: None
   status: 0
  success: True
        t: array([  0.00000000e+00,   3.15789474e-06,   6.31578947e-06,
         9.47368421e-06,   1.26315789e-05,   1.57894737e-05,
         1.89473684e-05,   2.21052632e-05,   2.52631579e-05,
         2.84210526e-05,   3.15789474e-05,   3.47368421e-05,
         3.78947368e-05,   4.10526316e-05,   4.42105263e-05,
         4.73684211e-05,   5.05263158e-05,   5.36842105e-05,
         5.68421053e-05,   6.00000000e-05])
 t_events: None
        y: array([[  1.00000000e-10,   2.77805305e-10,              nan,
                     nan,              nan,              nan,
                     nan,              nan,              nan,
                     nan,              nan,              nan,
                     nan,              nan,              nan,
                     nan,              nan,             

../pychemkin/chemkin.py:146: RuntimeWarning: overflow encountered in multiply
  pr[self.reversible] = pr[self.reversible] -  kb * np.product(x ** self.nu_prod[:, self.reversible], axis=0)
../pychemkin/chemkin.py:146: RuntimeWarning: invalid value encountered in subtract
  pr[self.reversible] = pr[self.reversible] -  kb * np.product(x ** self.nu_prod[:, self.reversible], axis=0)


In [107]:
from scipy.integrate import solve_ivp, ode
x_init = np.ones(4)*1.e-10
T = 1000
chem = chemkin("../tests/test_xml/rxns_singlereversible.xml")
cs = ChemSolver(chem)
_,cs.kf,cs.kb = cs.chem._progress_rate_init(x_init,T)
cs.nu_diff_matrix = cs.chem.nu_prod-cs.chem.nu_react
print(solve_ivp(cs._ChemSolver__dy_dt, 
          t_span=(0, 7e-5), 
          y0=x_init, 
          t_eval=np.linspace(0,7e-5,20), method = 'LSODA'))

print("Using scipy.integrate.ode")

r = ode(cs._ChemSolver__dy_dt).set_integrator('lsoda')
r.set_initial_value(x_init,0)
t1 = 7.0e-5
dt = 3.15789474e-06
while r.successful() and r.t <= t1:
    print(r.t+dt, r.integrate(r.t+dt))

Finished reading xml input file
  message: 'The solver successfully reached the interval end.'
     nfev: 44
     njev: 1
      nlu: 1
      sol: None
   status: 0
  success: True
        t: array([  0.00000000e+00,   3.68421053e-06,   7.36842105e-06,
         1.10526316e-05,   1.47368421e-05,   1.84210526e-05,
         2.21052632e-05,   2.57894737e-05,   2.94736842e-05,
         3.31578947e-05,   3.68421053e-05,   4.05263158e-05,
         4.42105263e-05,   4.78947368e-05,   5.15789474e-05,
         5.52631579e-05,   5.89473684e-05,   6.26315789e-05,
         6.63157895e-05,   7.00000000e-05])
 t_events: None
        y: array([[  1.00000000e-10,   1.46963082e-10,   1.84251428e-10,
          1.87352333e-10,   1.87836997e-10,   1.88003475e-10,
          1.88069885e-10,   1.88095959e-10,   1.88113270e-10,
          1.88122803e-10,   1.88127346e-10,   1.88129650e-10,
          1.88130882e-10,   1.88131525e-10,   1.88131854e-10,
          1.88132024e-10,   1.88132143e-10,   1.88132311e-10,


In [108]:
def get_equilibrium_conc(x_init, T):
    file_name = "../tests/test_xml/rxns_singlereversible.xml"
    input_ = InputParser(file_name)
    rc_list = [ReactionCoeffs(**params) for params in input_.rate_coeff_params]
    rc_list[0].set_params(T=T)
    kf = rc_list[0].k_forward()
    bc = BackwardCoeffs(input_.nu_react, input_.nu_prod,input_.species)
    kb = bc.backward_coeffs(np.array([kf]), T)[0]
    k_eq = kf/kb
    A = (1-k_eq)
    B = x_init[2]+x_init[3]+k_eq*(x_init[0]+x_init[1])
    C = x_init[2]*x_init[3]-k_eq*x_init[0]*x_init[1]
    deltax =  (-B+np.sqrt(B**2-4*A*C))/(2*A)
    return np.array([x_init[0]-deltax,x_init[1]-deltax,x_init[2]+deltax,x_init[3]+deltax])
    
get_equilibrium_conc(x_init, T)

Finished reading xml input file


array([  1.88132208e-10,   1.88132208e-10,   1.18677920e-11,
         1.18677920e-11])